In [53]:
## read in cleaned CSV's
import pandas as pd
import numpy as np
from dateutil.parser import parse

#import and read data
merged1 = pd.read_csv("/Users/inesschwartz/GreenDataScience/Thesis/tables_clean/merged1.csv")
samples1 = pd.read_csv("/Users/inesschwartz/Desktop/samples1.csv")
site_info1 = pd.read_csv("/Users/inesschwartz/Desktop/site_info1.csv")
morphology1 = pd.read_csv("/Users/inesschwartz/Desktop/morpho_cleaned.csv") #change once translated script ready
#profile_loc = pd.read_excel("")
#soil_profile = pd.read_excel("")


profile_record_clean1 = pd.read_csv("/Users/inesschwartz/Desktop/profile_record_clean.csv") # just matches profile record w profile, 'site_info_id', sample_id', and soil_type_id

#soil_type = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/Perfis_solo.xlsx")

In [2]:
#read in original csvs
import pandas as pd
import numpy as np
from dateutil.parser import parse

#import and read data
samples = pd.read_excel("/Users/inesschwartz/GreenDataScience/Thesis/tables_soil_database/AmostrasAngolaTerrario.xlsx")
analyses = pd.read_excel("/Users/inesschwartz/GreenDataScience/Thesis/tables_soil_database/Horizontes Analises.xlsx")
morphology = pd.read_excel("/Users/inesschwartz/GreenDataScience/Thesis/tables_soil_database/Horizontes_Morfologia.xlsx")
profile_loc = pd.read_excel("/Users/inesschwartz/GreenDataScience/Thesis/tables_soil_database/Perfis_local.xlsx")
soil_profile = pd.read_excel("/Users/inesschwartz/GreenDataScience/Thesis/tables_soil_database/Perfis_solo.xlsx")
elemental_analyses = pd.read_excel("/Users/inesschwartz/GreenDataScience/Thesis/tables_soil_database/Data XRF Angola_inicial.xlsx")
#soil_type = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/Perfis_solo.xlsx")

## Clean samples data (AmostrasAngolaTerrario)

In [10]:
# Rename columns for clarity
samples.rename(columns={
    'Registo': 'sample_id',
    'Nº Campo': 'site_info_id',
    'Ano': 'year',
    'Perfil': 'profile',
    'Campanha': 'campaign',
    'Colónia_Pais': 'country',
    'Distrito': 'district',
    'AmostraCrivada': 'sample_sifted',
    'AmostraNaoCrivada': 'sample_not_sifted',
    'Prateleira': 'shelf',
    'Sala': 'room'
}, inplace=True)

# Drop unused columns
samples_cleaning = samples.drop(columns=[
    'campaign', 'country', 'Província', 'sample_not_sifted', 'sample_sifted', 'Obs'
], errors='ignore')

# # Add a new Primary Key ID column starting from 1
# samples_cleaning.insert(0, 'sample_id', range(1, len(samples_cleaning) + 1))

# Add empty FK columns
samples_cleaning['lab_info_id'] = pd.NA
samples_cleaning['morpho_id'] = pd.NA
samples_cleaning['site_info_id'] = pd.NA

In [15]:
# check for duplicates, nulls, datatypes etc



In [11]:
samples_cleaning.head()

,sample_id,site_info_id,year,profile,district,shelf,room,lab_info_id,morpho_id
0,630,<NA>,1946.0,139,Huambo,1,22,<NA>,<NA>
1,631,<NA>,1946.0,139,Huambo,1,22,<NA>,<NA>
2,632,<NA>,1946.0,139,Huambo,1,22,<NA>,<NA>
3,633,<NA>,1946.0,139,Huambo,1,22,<NA>,<NA>
4,687,<NA>,1946.0,208,Huambo,1,22,<NA>,<NA>


In [20]:
# Ensure consistent types and formatting
samples_cleaning['profile'] = samples_cleaning['profile'].astype(str).str.replace('/', '_').str.strip().str[:20]
samples_cleaning['shelf'] = samples_cleaning['shelf'].astype("string")

#formatting site_info_id
samples_cleaning['site_info_id'] = samples_cleaning['site_info_id'].astype(str).str.replace('/', '_').str.strip().str[:20]
#formatting morpho_id
samples_cleaning['morpho_id'] = samples_cleaning['morpho_id'].astype(str).str.strip().str[:20]
#formatting lab_info_id
samples_cleaning['lab_info_id'] = samples_cleaning['lab_info_id'].astype(str).str.strip().str[:20]
# formatting year
samples_cleaning['year'] = samples_cleaning['year'].apply(
    lambda x: str(int(x)) if pd.notnull(x) else ''
)

# Convert sample_id to string (no truncation unless necessary)
samples_cleaning['sample_id'] = samples_cleaning['sample_id'].astype(str).str.strip()

# Check for duplicates AFTER conversion
duplicate_ids = samples_cleaning[samples_cleaning.duplicated('sample_id', keep=False)]
print("🔍 Duplicate sample_id values:")
print(duplicate_ids[['sample_id']])

samples_cleaning1 = samples_cleaning



🔍 Duplicate sample_id values:
Empty DataFrame
Columns: [sample_id]
Index: []


In [21]:
samples_cleaning1.dtypes

sample_id               object
site_info_id            object
year                    object
profile                 object
district                object
shelf           string[python]
room                    object
lab_info_id             object
morpho_id               object
dtype: object

In [22]:
# Reorder to match DB schema
samples_check = samples_cleaning1[[
    'sample_id',
    'site_info_id',
    'morpho_id',
    'lab_info_id',
    'profile',
    'year',
    'shelf',
    'room'
]]

samples_check.head()

,sample_id,site_info_id,morpho_id,lab_info_id,profile,year,shelf,room
0,630,<NA>,<NA>,<NA>,139,1946,1,22
1,631,<NA>,<NA>,<NA>,139,1946,1,22
2,632,<NA>,<NA>,<NA>,139,1946,1,22
3,633,<NA>,<NA>,<NA>,139,1946,1,22
4,687,<NA>,<NA>,<NA>,208,1946,1,22


In [23]:
samples_check.to_csv("/Users/inesschwartz/Desktop/samples_check.csv", index=False)

# Soil Profile Table

In [24]:
##rename columns for clarity add FK/PK
#create copy of samples
soil_profile_record = samples.copy()
# Rename columns in the DataFrame
samples.rename(columns={
    'Registo': 'sample_id',
    'Nº Campo': 'site_info_id',
    'Ano': 'year',
    'Perfil': 'profile',
    'Campanha': 'campaign',
    'Colónia_Pais': 'country',
    'Distrito': 'district',
    'AmostraCrivada': 'sample_sifted',
    'AmostraNaoCrivada': 'sample_not_sifted',
    'Prateleira': 'shelf',
    'Sala': 'room'
}, inplace=True)

soil_profile_cleaning2 = soil_profile_record
# Add missing columns
soil_profile_cleaning2['site_info_id'] = pd.NA
soil_profile_cleaning2['sample_id'] = pd.NA 
soil_profile_cleaning2['soil_type_id'] = pd.NA 
soil_profile_cleaning2['site_info_id'] = soil_profile_cleaning2['site_info_id'].astype(str)

# Add a new Primary Key ID column starting from 1
# Check if 'lab_sample_id' exists and drop it
if 'profile_record_id' in soil_profile_cleaning2.columns:
    soil_profile_cleaning2 = soil_profile_cleaning2.drop(columns=['profile_record_id'])

# Insert a new column, e.g., 'lab_sample_id' as a primary key starting from 1
soil_profile_cleaning2.insert(0, 'profile_record_id', range(1, len(soil_profile_cleaning2) + 1))

# Reorder columns to match SAMPLES schema
profile_record_clean = soil_profile_cleaning2[[
    'profile_record_id',
    'profile',
    'site_info_id',
    'sample_id',
    'soil_type_id'
]].copy()

In [25]:
# ensure consistent data types and formatting

# correct datatypes
#profile to string
profile_record_clean['profile'] = profile_record_clean['profile'].astype(str).str.strip().str[:20]

#sample_id to string
# Convert cleaned values to numeric safely
# Convert sample_id to numeric first (coerce errors to NaN)
profile_record_clean['sample_id'] = pd.to_numeric(profile_record_clean['sample_id'], errors='coerce')
# Then convert to string (will keep NaNs as <NA>)
profile_record_clean['sample_id'] = profile_record_clean['sample_id'].astype('Int64').astype(str)


In [26]:
# check for duplicates and nulls
# Filter out and print duplicated samples (including the first occurrence)
duplicated_values = profile_record_clean['sample_id'][profile_record_clean['sample_id'].duplicated()].unique()
print(duplicated_values)

['<NA>']


In [27]:
# Preview the result
profile_record_clean.head()
#profile_record_clean.to_csv("/Users/inesschwartz/Desktop/profile_record_clean.csv", index=False)

,profile_record_id,profile,site_info_id,sample_id,soil_type_id
0,1,139,<NA>,<NA>,<NA>
1,2,139,<NA>,<NA>,<NA>
2,3,139,<NA>,<NA>,<NA>
3,4,139,<NA>,<NA>,<NA>
4,5,208,<NA>,<NA>,<NA>


# Site info table

In [56]:
import unicodedata

site_info_cleaning = profile_loc

site_info_cleaning.rename(columns={
    'PERFIL': 'profile',
    'X_COORD': 'X_coord',
    'Y_COORD': 'Y_coord', 
    'PRO': 'district'
}, inplace=True)

# Add a new Primary Key ID column starting from 1
site_info_cleaning.insert(0, 'site_info_id', range(1, len(site_info_cleaning) + 1))


# Add missing columns
site_info_cleaning['land_cover_id'] = pd.NA
site_info_cleaning['climate_id'] = pd.NA  
site_info_cleaning['geology_id'] = pd.NA
site_info_cleaning['topo_feature_id'] = pd.NA  
site_info_cleaning['sampling_date'] = pd.NA # might leave out...
site_info_cleaning['districts_id'] = pd.NA  

# Function to remove accents
def remove_accents(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text)
        return ''.join(c for c in text if not unicodedata.combining(c))
    return text

# Apply to all object (string) columns
for col in site_info_cleaning.select_dtypes(include='object').columns:
    site_info_cleaning[col] = site_info_cleaning[col].apply(remove_accents)

# Replace / with _ and strip/shorten 'profile' BEFORE slicing into site_info_clean
site_info_cleaning['profile'] = (
    site_info_cleaning['profile']
    .astype(str)
    .str.replace('/', '_')
    .str.strip()
    .str[:20]
)

# Then select columns
site_info_clean = site_info_cleaning[[
    'site_info_id',
    'ID',
    'profile',
    'X_coord',
    'Y_coord',
    'land_cover_id',
    'climate_id',
    'geology_id',
    'topo_feature_id',
    'sampling_date',
    'districts_id',
    'district'
]]


site_info_clean.head()

,site_info_id,ID,profile,X_coord,Y_coord,land_cover_id,climate_id,geology_id,topo_feature_id,sampling_date,districts_id,district
0,1,2770,1_57,12.161278,-15.222598,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Namibe
1,2,48,1_59,12.575775,-4.866986,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Cabinda
2,3,1618,1_61,15.098840,-11.225411,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Cuanza Sul
3,4,881,1_63,17.081955,-9.274587,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Malanje
4,5,1750,1_64,20.788116,-11.568683,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Moxico


In [ ]:
site_info_clean.to_csv("/Users/inesschwartz/Desktop/site_info1.csv")

# Morphology Horizon Table

In [32]:
# rename columns
morphology.rename(columns={
    'Morfo_id':'horizon_id',
    'Amostra': 'sample_id',
    'Perfil': 'profile',
    'CM':'horizon_layer',
    'Limite Superior': 'upper_depth',
    'Limite inferior': 'lower_depth',
    'Grau de humidade': 'moisture_degree',
    'Quantidade de raízes': 'root_quantity',
    'Diâmetro de raízes': 'root_diameter',
    'Manchas': 'stains',
    'Textura': 'texture',
    'Abundância de elementos grosseiros': 'thick_contents_count',
    'Forma de elementos grosseiros': 'thick_contents_shape',
    'Natureza de elementos grosseiros': 'thick_contents_nature',
    'Tipo de estrutura': 'structure_type',
    'Classes de estrutura': 'structure_class',
    'Grau de estrutura': 'structure_degree',
    'Diâmetro de poros': 'pore_diameter',
    'Quantidade de poros': 'pore_quantity',
    'Forma de poros': 'pore_shape',
    'Cor (s)': 'dry_color_name',
    'Matiz (s)': 'dry_hue',
    'Valor (s)':'dry_value',
    'Croma (s)': 'dry_chroma',
    'Cor (h)': 'moist_color_name',
    'Matiz (h)': 'moist_hue',
    'Valor (h)': 'moist_value',
    'Croma (h)': 'moist_chroma',
    'Compacidade':'compaction',
    'Dureza': 'durability', 
    'Friabilidade': 'friability'
}, inplace=True)

# # Drop unnecessary columns
# morphology_cleaning = morphology.drop(columns=[
#     'ID1', 'Agrupamento', 'REF', 'Pro', 'Observaçoes', 'Horizonte de diagnóstico', 'Propriedade de diagnóstico', 'Nitidez do limite', 'Designação do horizonte', 'Observaçoes', 'Confirmar', 'Adesividade', 'Plasticidade', 'Efervescência com HCl', 'Friabilidade', 'Orientação das Fendas', 'Largura das fendas', 'Quantidade de fendas'
# ])

morphology_cleaning = morphology
#add profile_record_id to populate later from soil profile table
morphology_cleaning['profile_record_id'] = pd.NA

#drop accents
import unicodedata
def remove_accents(text):
    if isinstance(text, str):
        # Normalize and remove diacritics
        text = unicodedata.normalize('NFKD', text)
        text = ''.join(c for c in text if not unicodedata.combining(c))
        return text
    return text

# Apply to all cells in the DataFrame
morphology_cleaning = morphology_cleaning.applymap(remove_accents)

In [33]:
# Reorder columns to match SAMPLES schema
morphology_cleaning1 = morphology_cleaning [[
    'morpho_id',
    'sample_id',
    'profile_record_id',
    'profile',
    'horizon_layer',
    'upper_depth',
    'lower_depth',
    'moisture_degree',
    'root_quantity',
    'root_diameter',
    'texture',
    'structure_type',
    'structure_class',
    'structure_degree',
    'pore_diameter',
    'pore_quantity',
    'pore_shape',
    'dry_color_name',
    'dry_hue',
    'dry_value',
    'dry_chroma',
    'moist_color_name',
    'moist_hue',
    'moist_value',
    'moist_chroma',
    'compaction',
    'durability',
    'friability',
    'thick_contents_count',
    'thick_contents_nature'
]]

# Show first few rows
morphology_cleaning1.head(5)

,morpho_id,sample_id,profile_record_id,profile,horizon_layer,upper_depth,lower_depth,moisture_degree,root_quantity,root_diameter,...,dry_chroma,moist_color_name,moist_hue,moist_value,moist_chroma,compaction,durability,friability,thick_contents_count,thick_contents_nature
0,B_101/62_1_1,10999.0,<NA>,101/62,1.0,0.0,11.0,Seco,Muitas finas e bastantes medias,NaN,...,2.5,Pardo-acinzentado-escuro,10YR,4.0,2.0,Pequena a minima,Brando,NaN,NaN,NaN
1,B_101/62_2_1,11000.0,<NA>,101/62,2.0,11.0,28.0,Seco,Bastantes finas e medias e raras grossas,NaN,...,3.0,Pardo-amarelado-escuro,10YR,3.0,4.0,Pequena,Brando,NaN,NaN,NaN
2,B_101/62_3_1,11001.0,<NA>,101/62,3.0,28.0,54.0,Seco,Algumas finas e medias e raras grossas,NaN,...,4.0,Pardo-amarelado-escuro,10YR,4.0,4.0,Pequena a minima,Brando,NaN,NaN,NaN
3,B_101/62_4_1,11002.0,<NA>,101/62,4.0,54.0,90.0,Seco,"Poucas finas, algumas medias e raras grossas",NaN,...,6.0,Pardo-forte,"7,5YR",5.0,6.0,Pequena a minima,Brando,NaN,NaN,NaN
4,B_101/62_5_2,11003.2,<NA>,101/62,5.0,90.0,160.0,Seco a humido,Raras,Medias e grossas,...,6.0,Pardo-forte,"7,5YR",5.0,6.0,Pequena,Brando,NaN,NaN,NaN


In [39]:
# Ensure consistent data types and formatting
morphology_cleaning1['profile'] = morphology_cleaning1['profile'].astype(str).str.replace('/', '_').str.strip().str[:20]

#convert sample_to to string of the integer if it's a float like 11003.0
#The original string if it's not an integer or has decimal places
#"NULL" if the value is missing
def clean_sample_id(x):
    if pd.isnull(x):
        return "NULL"
    try:
        float_val = float(x)
        if float_val.is_integer():
            return str(int(float_val))
        else:
            return str(float_val)
    except:
        return str(x)

morphology_cleaning1['sample_id'] = morphology_cleaning1['sample_id'].apply(clean_sample_id)


/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_7811/4253896047.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morphology_cleaning1['profile'] = morphology_cleaning1['profile'].astype(str).str.replace('/', '_').str.strip().str[:20]
/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_7811/4253896047.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morphology_cleaning1['sample_id'] = morphology_cleaning1['sample_id'].apply(clean_sample_id)


In [41]:
# Check for nulls in 'sample_id' column
null_string_values = morphology_cleaning1[morphology_cleaning1['sample_id'] == "NULL"]['sample_id'].unique()
print(null_string_values)

null_string_count = (morphology_cleaning1['sample_id'] == "NULL").sum()
print(f"🔍 Number of sample_id values equal to 'NULL': {null_string_count}")

#check for nulls in morpho_id column
# Check for nulls in 'sample_id' column
null_string_values = morphology_cleaning1[morphology_cleaning1['morpho_id'] == "NULL"]['sample_id'].unique()
print(null_string_values)

null_string_count = (morphology_cleaning1['morpho_id'] == "NULL").sum()
print(f"🔍 Number of samorpho_idmple_id values equal to 'NULL': {null_string_count}")

['NULL']
🔍 Number of sample_id values equal to 'NULL': 6638
[]
🔍 Number of samorpho_idmple_id values equal to 'NULL': 0


In [47]:
# Check for nulls in 'sample_id' column
null_string_values = morphology_cleaning[morphology_cleaning1['sample_id'] == "NULL"]['sample_id'].unique()
print(null_string_values)

null_string_count = (morphology_cleaning1['sample_id'] == "NULL").sum()
print(f"🔍 Number of sample_id values equal to 'NULL': {null_string_count}")


['nan' '4948' '4949']
🔍 Number of sample_id values equal to 'NULL': 6638


/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_7811/3405257731.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  null_string_values = morphology_cleaning[morphology_cleaning1['sample_id'] == "NULL"]['sample_id'].unique()


**Goal** = drop any morpho_id from morphology_cleaning that does not have both:

1. Key horizon characteristics

2. A valid profile (w location and site characteristics) that appears in site_info_clean

In [57]:
import pandas as pd

# -----------------------------------------
# 1. Standardize 'profile' column formatting
# -----------------------------------------
site_info_clean.loc[:, 'profile'] = (
    site_info_clean['profile']
    .astype(str)
    .str.replace('/', '_')
    .str.strip()
    .str.upper()
    .str[:20]
)

morphology_cleaning1.loc[:, 'profile'] = (
    morphology_cleaning1['profile']
    .astype(str)
    .str.replace('/', '_')
    .str.strip()
    .str.upper()
    .str[:20]
)

# ---------------------------
# 2. Define valid profiles
# ---------------------------
valid_profiles = set(site_info_clean['profile'])

# ---------------------------
# 3. Define helper to check for missing/zero
# ---------------------------
def is_missing(val):
    return pd.isna(val) or val == 0 or val == '0'

# ---------------------------
# 4. Build masks for filtering
# ---------------------------

# A. Missing key horizon data
missing_key_data_mask = (
    morphology_cleaning1['morpho_id'].notna() &
    morphology_cleaning1['moist_color_name'].apply(is_missing) &
    morphology_cleaning1['texture'].apply(is_missing) &
    morphology_cleaning1['structure_type'].apply(is_missing)
)

# B. Invalid profile
invalid_profile_mask = ~morphology_cleaning1['profile'].isin(valid_profiles)

# C. Missing both upper and lower depth
missing_depth_mask = (
    morphology_cleaning1['upper_depth'].apply(is_missing) &
    morphology_cleaning1['lower_depth'].apply(is_missing)
)

# ---------------------------
# 5. Combine all masks
# ---------------------------
rows_to_flag_mask = missing_key_data_mask | invalid_profile_mask | missing_depth_mask
rows_to_flag = morphology_cleaning[rows_to_flag_mask]

# ---------------------------
# 6. Report and drop flagged rows
# ---------------------------
print(f"⚠️ Found {len(rows_to_flag)} rows with at least one of the following issues:")
print("   • missing key horizon data")
print("   • invalid profile (not in site_info_clean)")
print("   • missing both upper_depth and lower_depth")

# Optional: Save flagged rows for review
# rows_to_flag.to_csv("/Users/inesschwartz/Desktop/morph_flagged_rows_combined.csv", index=False)

# Drop flagged rows from main DataFrame
morphology_cleaning1 = morphology_cleaning1[~rows_to_flag_mask].copy()

# Summary after cleanup
print(f"✅ Remaining rows in morphology_cleaning: {len(morphology_cleaning1)}")


⚠️ Found 2589 rows with at least one of the following issues:
   • missing key horizon data
   • invalid profile (not in site_info_clean)
   • missing both upper_depth and lower_depth
✅ Remaining rows in morphology_cleaning: 10434


In [58]:
# check for duplicates
# Make sure sample_id and horizon_id are strings
morphology_cleaning1['sample_id'] = morphology_cleaning1['sample_id'].astype(str).str.strip()
morphology_cleaning1['morpho_id'] = morphology_cleaning1['morpho_id'].astype(str).str.strip()
# Filter out and print duplicated samples (including the first occurrence)
duplicated_values = morphology_cleaning1['sample_id'][morphology_cleaning1['sample_id'].duplicated()].unique()
print(duplicated_values)

['NULL' '8643']


In [59]:
#addressing duplicates

# -----------------------------------
# 1. Correct sample_id using morpho_id map
# -----------------------------------
# Step 1: Make sure IDs are strings and trimmed
morphology_cleaning1['morpho_id'] = morphology_cleaning1['morpho_id'].astype(str).str.strip()
morphology_cleaning1['sample_id'] = morphology_cleaning1['sample_id'].astype(str).str.strip()

# Step 2: Your mapping (morpho_id → new sample_id)
horizon_to_sample_map = {
    'UZ_24c/60_1_1': '8639',
    'UZ_24c/60_2_1': '8640',
    'UZ_24c/60_3_1': '8641',
    'UZ_24c/60_4_1': '8642',
    'UZ_24c/60_5_1': '8643',
    'UZ_66c/60_1_1': '8777',
    'UZ_66c/60_2_1': '8778',
    'UZ_66c/60_3_1': '8779',
    'UZ_66c/60_4_1': '8780',
    'CC_473/66_1_1': '17596',
    'CC_473/66_2_1': '17597',
    'CC_473/66_3_1': '17598',
    'N_85/57_1_0': '4947',
    'N_85/57_2_0': '4948',
    'N_85/57_3_0': '4949',
    'UZ_231/60_4_1': '8255',
    'B_139/61_5_1': '10353',
    'H_480/55_1_1': '3919',
    'H_91/54_1_1': '1255',
    'H_90/54_3_3': '1252',
    'Hb_136/56_1_1':'2961',
    'Hb_136/56_2_1': '2962',
    'Hb_136/56_3_1': '2963',
    'Hb_136/56_4_1': '2964',
    'UZ_216/60_5_1': '8231',
    'Bg_217/46_1_1': '694',
    'Bg_217/46_2_1': '695',
    'Bg_217/46_3_1': '696',
    'UZ_215c/60_1_1':'9255',
    'UZ_215c/60_2_1': '9256',
    'UZ_215c/60_3_1': '9257',
    'UZ_215c/60_4_1': '9258',
    'UZ_215c/60_5_1': '9259',
    'UZ_215c/60_6_1': '9260',
    'N_30/57_1_1':'4848',
    'N_30/57_2_2':'4849',
    'N_30/57_3_1':'4850',
    'UZ_66c/60_1_1': '8777',
    'UZ_66c/60_2_1': '8778',
    'UZ_66c/60_3_1': '8779',
    'UZ_66c/60_4_1': '8780',
    'UZ_66c/60_5_1': '8781',
    'UZ_66c/60_6_1': '8783',
    'UZ_66c/60_7_1': '8782',
    'H_91/54_1_1': '1255',
    'H_91/54_2_1': '1256',
    'H_91/54_3_1': '1257'
}

# Step 3: Apply the mapping to replace sample_id based on horizon_id
morphology_cleaning1['sample_id'] = morphology_cleaning1.apply(
    lambda row: horizon_to_sample_map.get(row['morpho_id'], row['sample_id']),
    axis=1
)

# -----------------------------------
# 2. Drop specific horizon_ids (might have already been dropped)
# -----------------------------------
horizons_to_drop = [
    'B_139/61_4_1', 'H_113/55_1_1', 'H_1306/52_1_1', 'H_1611/52_1_1',
    'H_227/46_1_1', 'H_227/46_2_1', 'H_227/46_3_1', 'H_227/46_4_1',
    'Mj_178c/63_4_1', 'Mj_321c/63_4_1', 'B_110/62_5_1', 'B_110/62_6_1'
]
morphology_cleaning1 = morphology_cleaning1[~morphology_cleaning1['morpho_id'].isin(horizons_to_drop)]


# -----------------------------------
# 3. Done — Optional: Save to file
# -----------------------------------
#df_morpho_cleaned.to_csv("/Users/inesschwartz/Desktop/df_morpho_cleaned_corrected.csv", index=False)
print("✅ Morphology data cleaned and sample_id updated via morpho_id.")


✅ Morphology data cleaned and sample_id updated via morpho_id.


In [60]:
#re-count null sample_id's
morphology_cleaning1
# Check for nulls in 'sample_id' column
null_string_values2 = morphology_cleaning1[morphology_cleaning1['sample_id'] == "NULL"]['sample_id'].unique()
print(null_string_values2)

null_string_count2 = (morphology_cleaning1['sample_id'] == "NULL").sum()
print(f"🔍 Number of sample_id values equal to 'NULL': {null_string_count2}")

['NULL']
🔍 Number of sample_id values equal to 'NULL': 4713


In [61]:
num_duplicates = morphology_cleaning1['morpho_id'].duplicated().sum()
print(f"🔁 Found {num_duplicates} duplicated morpho_id(s).")


🔁 Found 0 duplicated morpho_id(s).


In [62]:
df_morpho_cleaned = morphology_cleaning1

### dealing w decimals in sample_id

In [63]:
# Step 1: Ensure sample_id is a string
df_morpho_cleaned['sample_id'] = df_morpho_cleaned['sample_id'].astype(str).str.strip()

# Step 2: Identify sample_ids that contain a decimal point
decimal_sample_ids = df_morpho_cleaned[df_morpho_cleaned['sample_id'].str.contains(r'\.\d+$', regex=True)]

# Step 3: Output results
print(f"🔍 Found {len(decimal_sample_ids)} sample_id(s) with decimal values.\n")

# Optional: Print the unique sample_ids with decimals
print("🧾 Affected sample_ids:")
print(decimal_sample_ids['sample_id'].unique())


🔍 Found 439 sample_id(s) with decimal values.

🧾 Affected sample_ids:
['11003.2' '11285.2' '10843.2' '10428.2' '10470.2' '10480.2' '10485.2'
 '10446.2' '10849.2' '10509.2' '10562.2' '10536.2' '11327.2' '11356.2'
 '11393.2' '11176.2' '11454.2' '10885.2' '10904.2' '10922.2' '11485.2'
 '10933.2' '4972.3' '4981.2' '5810.2' '5045.2' '5822.2' '4989.2' '5006.2'
 '5010.2' '5839.2' '4996.2' '5082.2' '5026.2' '6314.2' '6317.2' '6323.2'
 '6332.2' '6336.3' '5883.2' '6342.3' '6348.2' '6350.2' '6358.2' '5902.2'
 '5897.3' '6376.3' '6381.2' '6383.3' '6388.2' '6391.2' '6402.2' '6405.2'
 '6416.2' '5914.2' '5916.2' '6439.3' '5088.2' '5053.2' '5101.2' '5107.2'
 '5123.2' '5945.2' '5174.2' '5176.2' '5963.2' '5973.2' '5202.2' '5207.2'
 '5192.2' '5214.2' '5218.2' '5225.3' '5983.2' '5990.2' '5270.2' '5995.2'
 '5997.2' '5447.2' '6014.2' '6022.2' '5417.2' '5332.2' '6032.2' '5476.2'
 '5478.2' '5434.2' '5359.2' '5442.2' '5444.2' '5327.2' '5342.2' '5315.2'
 '5348.2' '7390.2' '7397.2' '5717.2' '5745.2' '5751.2' '573

In [64]:
# save to csv
df_morpho_cleaned.to_csv("/Users/inesschwartz/Desktop/morpho_cleaned_updated.csv")

# Soil Type Table

In [65]:
# Rename columns
soil_type = soil_profile.copy()

soil_type.rename(columns={
    'Perfil': 'profile',
    'Agrupamento': 'grouping',
    'Pro': 'province',
    'País': 'country',
    'Local': 'location',
    'DATA': 'date',
    'CEP_NOME': 'CEP_NAME',
}, inplace=True)

# Drop unnecessary columns
soil_type_cleaning = soil_type.drop(columns=[
    'REF', 'province', 'country', 'location', 'DESCRITOR1', 'DESCRITOR2', 'DESCRITOR3',
    'date', 'Fase', 'D_INSERÇAO', 'Publicação', 'WRB_old', 'Missão'
], errors='ignore')  # use errors='ignore' in case some columns were already missing

# Add a new Primary Key ID column starting from 1
soil_type_cleaning.insert(0, 'soil_type_id', range(1, len(soil_type_cleaning) + 1))

# Drop accents
import unicodedata

def remove_accents(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text)
        return ''.join(c for c in text if not unicodedata.combining(c))
    return text

# Apply to all cells in the DataFrame
soil_type_cleaning = soil_type_cleaning.applymap(remove_accents)

# Replace / with _ and strip/shorten 'profile' BEFORE slicing into site_info_clean
soil_type_cleaning['profile'] = (
    soil_type_cleaning['profile']
    .astype(str)
    .str.replace('/', '_')
    .str.strip()
    .str[:20]
)

# Keep only relevant columns
soil_type_clean = soil_type_cleaning[[
    'soil_type_id',
    'profile',
    'CEP_GR',
    'CEP_NAME',
    'FAO'
]]

# Preview
soil_type_clean.head(2)


,soil_type_id,profile,CEP_GR,CEP_NAME,FAO
0,1,1_51,NaN,NaN,NaN
1,2,1_57,Aridicos,Aridicos com calcario Pardo-cinzentos,CLha


In [66]:
# check duplicates
# Filter out and print duplicated samples (including the first occurrence)
duplicated_values_st = soil_type_clean['profile'][soil_type_clean['profile'].duplicated()].unique()
print(duplicated_values_st)

# Check for profile values that are the string "NULL"
null_string_values_st = soil_type_clean[soil_type_clean['profile'] == "NULL"]['profile'].unique()
print("🔍 Unique 'profile' values equal to string 'NULL':", null_string_values_st)

null_string_count_st = (soil_type_clean['profile'] == "NULL").sum()
print(f"🔢 Number of 'profile' values equal to string 'NULL': {null_string_count_st}")



[]
🔍 Unique 'profile' values equal to string 'NULL': []
🔢 Number of 'profile' values equal to string 'NULL': 0


In [67]:
# save clean table
soil_type_clean.to_csv("/Users/inesschwartz/Desktop/soil_type_clean.csv", index=False)

# Climate table


In [72]:
# Load original Excel data
profile_loc = pd.read_excel("/Users/inesschwartz/GreenDataScience/Thesis/tables_soil_database/Perfis_local.xlsx")

# Create a copy for cleaning
climate_features_cleaning = profile_loc.copy()

# Mapping from climate codes to descriptions
code_to_description = {
    "B1": "Húmido",
    "B2": "Húmido",
    "B3": "Húmido",
    "B4": "Húmido",
    "C1": "Sub-húmido seco",
    "C2": "Sub-húmido chuvoso",
    "D": "Semi-árido",
    "E": "Árido",
    "Aw": "Tropical chuvoso com estação seca no Inverno, de savana",
    "BSw": "Seco de estepe, com chuva predominante no Verão",
    "BWw": "Seco de deserto, com chuva predominante no Verão",
    "Cw": "Mesotérmico húmido com estação seca no Inverno",
    "ARe": "Arídico extremo",
    "ARf": "Arídico fraco",
    "ARt": "Arídico típico",
    "tUDs": "Tempúdico seco",
    "tUSh": "Tempústico húmido",
    "tUSt": "Tempústico típico",
    "TUDs": "Tropúdico seco",
    "TUSa": "Tropústico arídico",
    "TUSu": "Tropústico údico",
    "TUSt": "Tropustico típico",
    "H": "Hipertérmico",
    "iH": "Iso-Hipertérmico",
    "iT": "Iso-Térmico",
    "T": "Térmico"
}

# Replace climate codes with descriptions
for col in ["CL_THORNTH", "CL_KOPPEN", "REG_HÍDRIC", "REG_TÉRMIC"]:
    climate_features_cleaning[col] = climate_features_cleaning[col].replace(code_to_description)

# Function to average values like "21-22" -> 21.5
def average_range(value):
    if isinstance(value, str) and '-' in value:
        try:
            nums = [float(x.strip()) for x in value.split('-')]
            return sum(nums) / len(nums)
        except:
            return None
    try:
        return float(value)
    except:
        return None

# Apply to temperature and precipitation columns
climate_features_cleaning["TMA"] = climate_features_cleaning["TMA"].apply(average_range)
climate_features_cleaning["PMA"] = climate_features_cleaning["PMA"].apply(average_range)

# Rename columns for clarity
climate_features_cleaning.rename(columns={
    'PERFIL': 'profile',
    'CL_THORNTH': 'thornthwaite_climate',
    'CL_KOPPEN': 'koppen_climate',
    'TMA': 'mean_annual_temp',
    'PMA': 'mean_annual_precip',
    'REG_HÍDRIC': 'hydric_regime',
    'REG_TÉRMIC': 'thermal_regime'
}, inplace=True)

# ensure consistent profile formatting
climate_features_cleaning['profile'] = climate_features_cleaning['profile'].astype(str).str.replace('/', '_').str.strip().str[:20]

# Drop accents from text values
def remove_accents(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text)
        return ''.join(c for c in text if not unicodedata.combining(c))
    return text

climate_features_cleaning = climate_features_cleaning.applymap(remove_accents)

# Add primary key column
climate_features_cleaning.insert(0, 'climate_id', range(1, len(climate_features_cleaning) + 1))

# Replace empty strings or nulls in numeric columns with \N (Postgres null)
for col in ['mean_annual_temp', 'mean_annual_precip']:
    climate_features_cleaning[col] = climate_features_cleaning[col].replace(
        ['', ' ', 'NULL', None, pd.NA, pd.NaT, 'nan', float('nan')], ''
    )

# Select and reorder relevant columns for export
climate_features_clean = climate_features_cleaning[[
    'climate_id',
    'profile',
    'ID',
    'mean_annual_temp',
    'mean_annual_precip',
    'koppen_climate',
    'thornthwaite_climate',
    'hydric_regime',
    'thermal_regime'
]]

# Preview
climate_features_clean.head()


,climate_id,profile,ID,mean_annual_temp,mean_annual_precip,koppen_climate,thornthwaite_climate,hydric_regime,thermal_regime
0,1,1_57,2770,,,NaN,Arido,NaN,NaN
1,2,1_59,48,,,NaN,Humido,NaN,NaN
2,3,1_61,1618,,,NaN,NaN,NaN,NaN
3,4,1_63,881,21.5,1500.0,"Tropical chuvoso com estacao seca no Inverno, ...",Humido,Tropustico udico,Iso-Hipertermico
4,5,1_64,1750,,,NaN,NaN,NaN,NaN


Check FK relationships and datatypes

In [73]:
# Check and print the datatypes of each column
print("Data types of each column in profile_clean:\n")
print(climate_features_clean.dtypes)

Data types of each column in profile_clean:

climate_id               int64
profile                 object
ID                       int64
mean_annual_temp        object
mean_annual_precip      object
koppen_climate          object
thornthwaite_climate    object
hydric_regime           object
thermal_regime          object
dtype: object


# Topo features table

In [74]:
# Create a copy for cleaning
topo_features_cleaning = profile_loc.copy()

# Rename columns for clarity
topo_features_cleaning.rename(columns={
    'PERFIL': 'profile',
    'TOPOGRAFIA': 'slope_code',
    'ALTITUDE': 'altitude',
}, inplace=True)

# ensure consistent profile formatting
topo_features_cleaning['profile'] = topo_features_cleaning['profile'].astype(str).str.replace('/', '_').str.strip().str[:20]

# Add missing columns
topo_features_cleaning['aspect'] = pd.NA  
topo_features_cleaning['land_surface_temp'] = pd.NA
topo_features_cleaning['dem_elevation'] = pd.NA  

# Add primary key column
topo_features_cleaning.insert(0, 'topo_features_id', range(1, len(topo_features_cleaning) + 1))

# Create slope class mapping dictionary
slope_code_to_description = {
    "D1": "Plano (Declives < 2%)",
    "D2": "Ondulado muito suave (Declives > 2% e < 3%)",
    "D3": "Ondulado suave (Declives > 3% e < 5%)",
    "D4": "Ondulado (Declives > 5% e < 8%)",
    "D5": "Acidentado (Declives > 8% e < 15%)",
    "D6": "Escarpado (Declives >15% e < 30%)",
    "D7": "Montanhoso (Declives > 30%)"
}

# Create a mapping DataFrame for slope classes
slope_classes_df = pd.DataFrame([
    {"slope_code": code, "slope_description": desc}
    for code, desc in slope_code_to_description.items()
])

#drop accents
import unicodedata
def remove_accents(text):
    if isinstance(text, str):
        # Normalize and remove diacritics
        text = unicodedata.normalize('NFKD', text)
        text = ''.join(c for c in text if not unicodedata.combining(c))
        return text
    return text

# Apply to all cells in the DataFrame
topo_features_cleaning = topo_features_cleaning.applymap(remove_accents)

# Final cleaned topo features table (referencing slope_code, not description)
topo_features_clean = topo_features_cleaning[[
    'topo_features_id',
    'ID',
    'profile', 
    'slope_code',
    'altitude',
    'aspect',
    'land_surface_temp',
    'dem_elevation'
]]

# Preview
topo_features_clean.head()


,topo_features_id,ID,profile,slope_code,altitude,aspect,land_surface_temp,dem_elevation
0,1,2770,1_57,NaN,32.0,<NA>,<NA>,<NA>
1,2,48,1_59,D5,NaN,<NA>,<NA>,<NA>
2,3,1618,1_61,NaN,NaN,<NA>,<NA>,<NA>
3,4,881,1_63,D1,1210.0,<NA>,<NA>,<NA>
4,5,1750,1_64,NaN,NaN,<NA>,<NA>,<NA>


# Geological features table

In [75]:
# Load cleaned geo features data
geo_features_cleaning = pd.read_excel("/Users/inesschwartz/GreenDataScience/Thesis/tables_soil_database/Perfis_local.xlsx")

# Rename columns
geo_features_cleaning.rename(columns={
    'PERFIL': 'profile',
    'GEOLOGIA': 'geology_id',
    'LITOLOGIA': 'lithology_id',
    'LITOLOGIA_1954': 'lithology_1954_id',
}, inplace=True)

# Add primary key column
geo_features_cleaning.insert(0, 'geo_features_id', range(1, len(geo_features_cleaning) + 1))

# Final normalized geo_features table (with codes as foreign keys)
geo_features_clean = geo_features_cleaning[[
    'geo_features_id',
    'profile',
    'ID',
    'geology_id',
    'lithology_id',
    'lithology_1954_id'
]]

# ensure consistent profile formatting
geo_features_clean['profile'] = geo_features_clean['profile'].astype(str).str.replace('/', '_').str.strip().str[:20]

#drop accents
import unicodedata
def remove_accents(text):
    if isinstance(text, str):
        # Normalize and remove diacritics
        text = unicodedata.normalize('NFKD', text)
        text = ''.join(c for c in text if not unicodedata.combining(c))
        return text
    return text

# Apply to all cells in the DataFrame
geo_features_clean = geo_features_clean.applymap(remove_accents)

#preview
geo_features_clean.head()

/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_2227/2818143663.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geo_features_clean['profile'] = geo_features_clean['profile'].astype(str).str.replace('/', '_').str.strip().str[:20]


,geo_features_id,profile,ID,geology_id,lithology_id,lithology_1954_id
0,1,1_57,2770,NaN,d,NaN
1,2,1_59,48,Oendolongo,pp,Sistema do Maiombe
2,3,1_61,1618,NaN,NaN,NaN
3,4,1_63,881,Karroo,Cs/Cal,Serie de Cassanje - T2'T1
4,5,1_64,1750,NaN,NaN,NaN


In [76]:
## mapping tables
# Mappings for geology
geology_mapping = {
    "Kalahari": "Sistema do Kalahari",
    "Superficiais": "Formações Superficiais",
    "Karroo": "Sistema do Karroo",
    "Bembe": "Sistema do Bembe",
    "Oendolongo": "Sistema do Oendolongo",
    "Base": "Complexo de base",
    "Proterozóico": "Proterozóico",
    "Pleistocénico": "Pleistocénico",
    "Terciário": "Terciário (médio e inferior)",
    "TQ": "Quaternário e Terciário superior",
    "Cretácio": "",  # no description provided
    "RPKS": "Recente Plistocénico e Kalahari Superior"
}

# Mappings for lithology_1954
lithology_1954_mapping = {
    "γ": "Granitos, Granodioritos e Quartzodioritos",
    "PL": "Xistos, metaquartzitos, conglomerados, arcoses, ect.",
    "λ": "Rochas eruptivas indeterminadas",
    "δp": "Doleritos, doleritos pigeoníticos",
    "δab": "Diabases, diabases albito-cloriticas",
    "ε": "Noritos, gabros e peridotitos",
    "JK": "Composto de conglomerados, areias, cascalhos do Kalahari",
    "C": "Série Xisto - calcária",
    "Cal": "Sedimentos arenosos não consolidados",
    "K": "Série xisto - gresosa",
    "RT": "Não diferenciado",
    "σ": "Sienitos, sienitos nefelínicos",
    "Q": "Depósitos fossilíferos",
    "CS": "Grande conglomerado e série de Mwashya"
}

# Mappings for lithology
lithology_mapping = {
    "a": "Rochas arenáceas consolidadas",
    "aq": "Grés quartzíticos do Oendolongo",
    "b": "Rochas eruptivas básicas",
    "c": "Rochas sedimentares consolidadas calcárias",
    "c'": "Rochas sedimentares não consolidadas calcárias",
    "cg": "Rochas cristalofílicas argiláceas",
    "d": "Sedimentos não consolidados de origem marinha",
    "dc": "Depósitos coluvionares",
    "dr": "Diorítos",
    "e": "Rochas sedimentares consolidadas não calcárias",
    "g": "Rochas argiláceas consolidadas não calcárias",
    "g'": "Rochas argiláceas não consolidadas não calcárias",
    "g''": "Rochas cristalinas pouco micas em quartzo",
    "gp": "Rochas do  complexo gabro-plagioclastíco",
    "k": "Sedimentos não consolidados grosseiros do Kalahari",
    "m": "Rochas sedimentares não consolidadas calco-gipsíferas",
    "m'": "Depósitos coluvionares margosos",
    "mm": "Materiais mistos",
    "n": "Sedimentos não consolidados de origem continental",
    "nd": "não descrito",
    "pp": "Sedimentos não consolidados grosseiros plio-plistocénicos",
    "q": "Rochas cristalinas quartzíferas",
    "q'": "Materiais redistribuídos provenientes de desagregação rochas crist. quartzíferas",
    "qf": "Quartzitos ferruginosos do Oendolongo",
    "r": "Sedimentos grosseiros não especificados",
    "s": "Sienitos",
    "sx": "Formações (ou rochas) sedimentares não especificadas",
    "sx1": "Rochas sedimentares consolidadas com e sem calcário",
    "sx2": "Rochas sedimentares consolidadas",
    "v": "Materiais vulcânicos",
    "v'": "Rochas do complexo alcalino e/ou carboatítico",
    "x": "Rochas consolidadas não especificadas",
    "xm": "Xistos metamórficos",
    "xq": "Rochas cristalinas não especificadas",
    "z": "Rochas metassedimentares"
}

# Save each mapping as a DataFrame
pd.DataFrame([
    {"geology_code": k, "geology_description": v}
    for k, v in geology_mapping.items()
]).to_csv("/Users/inesschwartz/GreenDataScience/Thesis/tables_clean/geology_mapping.csv", index=False)

pd.DataFrame([
    {"lithology_code": k, "lithology_description": v}
    for k, v in lithology_mapping.items()
]).to_csv("/Users/inesschwartz/GreenDataScience/Thesis/tables_clean/lithology_mapping.csv", index=False)

pd.DataFrame([
    {"lithology_1954_code": k, "lithology_1954_description": v}
    for k, v in lithology_1954_mapping.items()
]).to_csv("/Users/inesschwartz/GreenDataScience/Thesis/tables_clean/lithology1954_mapping.csv", index=False)


# District table

**don't need**

Why do I need to make a separate table??

# Minerology info table

# Biology Info table

# Checking CSV Datatypes before DB export

In [77]:
import pandas as pd
import os

# Set the path to your folder containing the 10 CSV files
csv_folder = "/Users/inesschwartz/GreenDataScience/Thesis/tables_clean"  

# List all CSV files in the folder
csv_files = [f for f in os.listdir(csv_folder) if f.endswith(".csv")]

# Loop through each file and display column data types
for file in csv_files:
    file_path = os.path.join(csv_folder, file)
    print(f"\n📄 File: {file}")
    
    try:
        df = pd.read_csv(file_path)
        print(df.dtypes)
    except Exception as e:
        print(f"⚠️ Error reading {file}: {e}")


📄 File: analyses.csv
lab_sample_id      int64
sample_id        float64
EG               float64
thick_clay       float64
fine_clay        float64
                  ...   
Tl               float64
Pb               float64
Bi               float64
Th               float64
U                float64
Length: 68, dtype: object

📄 File: soil_type_clean.csv
soil_type_id     int64
profile         object
CEP_GR          object
CEP_NAME        object
FAO             object
dtype: object

📄 File: analyses_clean.csv
lab_sample_id      int64
analysis_id       object
horizon_id        object
sample_id         object
EG               float64
                  ...   
Tl               float64
Pb               float64
Bi               float64
Th               float64
U                float64
Length: 70, dtype: object

📄 File: profile_record.csv
profile_record_id      int64
profile               object
site_info_id         float64
soil_type_id         float64
sample_id              int64
dtype: object

📄 

/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_2227/3654665737.py:16: DtypeWarning: Columns (13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


horizon_id            object
sample_id            float64
profile_record_id    float64
horizon_layer        float64
upper_depth          float64
lower_depth          float64
moisture_degree       object
root_quantity         object
root_diameter         object
texture               object
structure_type        object
structure_class       object
structure_degree      object
pore_diameter         object
pore_quantity         object
pore_shape            object
dry_color_name        object
dry_hue               object
dry_value            float64
dry_chroma           float64
moist_color_name      object
moist_hue             object
moist_value          float64
moist_chroma         float64
compaction            object
durability            object
dtype: object

📄 File: site_info.csv
site_info_id          int64
profile              object
X_coord             float64
Y_coord             float64
district             object
geo_features_id       int64
climate_id            int64
topo_features

/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_2227/3654665737.py:16: DtypeWarning: Columns (13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


# Foreign key imports and datatype consistency

In [78]:
## standardize datatypes for identifiers
def convert_identifiers_to_string(df, id_columns):
    """
    Converts specified identifier columns in a DataFrame to string type,
    safely handling float64 values and preserving missing values (NA).
    Avoids SettingWithCopyWarning.
    """
    df = df.copy()  # ensure we're working with a copy, not a slice
    for col in id_columns:
        if col in df.columns:
            df.loc[:, col] = df[col].apply(
                lambda x: str(int(x)) if pd.notna(x) and isinstance(x, float) and x.is_integer()
                else str(x) if pd.notna(x)
                else pd.NA
            ).astype("string")
    return df

#usage

# Define identifier columns
identifier_columns = [
    'sample_id', 'site_info_id', 'profile',
    'horizon_id', 'lab_sample_id', 'lab_sample_id', 'climate_id', 'geo_features_id', 'topo_features_id', 'profile_record_id'
]

# Apply to each relevant dataframe
samples_check = convert_identifiers_to_string(samples_check, identifier_columns)
df_morpho_cleaned = convert_identifiers_to_string(df_morpho_cleaned, identifier_columns)
profile_clean = convert_identifiers_to_string(profile_record_clean, identifier_columns)
merged1 = convert_identifiers_to_string(merged1, identifier_columns)
site_info_clean = convert_identifiers_to_string(site_info_clean, identifier_columns)
#district_clean = convert_identifiers_to_string(district_clean, identifier_columns)
soil_type_clean = convert_identifiers_to_string(soil_type_clean, identifier_columns)
geo_features_clean = convert_identifiers_to_string(geo_features_clean, identifier_columns)
topo_features_clean = convert_identifiers_to_string(topo_features_clean, identifier_columns)


In [79]:
# Check types of identifier columns in each dataframe
dfs = {
    "samples": samples_check,
    "horizon": df_morpho_cleaned,
    "profile": profile_clean,
    "lab_analysis": merged1,
    "site_info": site_info_clean,
    #"district_clean": district_clean,
    "soil_type": soil_type_clean,
    "geo_features": geo_features_clean,
    "topo_features": topo_features_clean,
}

for name, df in dfs.items():
    print(f"\n{name} column types:")
    for col in identifier_columns:
        if col in df.columns:
            print(f"  {col}: {df[col].dtype}")



samples column types:
  sample_id: object
  site_info_id: object
  profile: object
  horizon_id: object
  profile_record_id: object

horizon column types:
  sample_id: object
  profile: object
  horizon_id: object
  profile_record_id: object

profile column types:
  sample_id: object
  site_info_id: object
  profile: object
  profile_record_id: string

lab_analysis column types:
  sample_id: object
  horizon_id: object
  lab_sample_id: string
  lab_sample_id: string

site_info column types:
  site_info_id: string
  profile: object
  climate_id: object

soil_type column types:
  profile: object

geo_features column types:
  profile: object
  geo_features_id: string

topo_features column types:
  profile: object
  topo_features_id: string


In [80]:
## validate FK references in tables

samples_check['sample_id'] = samples_check['sample_id'].astype(str).str.strip().str.replace('.0', '')
df_morpho_cleaned['sample_id'] = df_morpho_cleaned['sample_id'].astype(str).str.strip().str.replace('.0', '')

# Check for missing FK references in morphology
missing_samples = samples_check[~samples_check['sample_id'].isin(df_morpho_cleaned['sample_id'])]

print(f"{len(missing_samples)} sample_id(s) in samples_check are missing in horizon.")
print(missing_samples[['sample_id']])


9704 sample_id(s) in samples_check are missing in horizon.
      sample_id
1           631
3           633
7           697
8           698
9           699
...         ...
14710     18867
14711     18868
14712     18869
14713     18870
14714     18871

[9704 rows x 1 columns]


In [81]:
#inspecting a random 25 samples that are missing in horizon

# Print random sample of 25 missing sample_ids
sample_subset = missing_samples.sample(n=25, random_state=42)  # Set random_state for reproducibility
print(sample_subset[['sample_id']].reset_index(drop=True))



   sample_id
0       2044
1      15936
2      17780
3       5288
4      13422
5      10276
6      12228
7       9094
8      15374
9       7359
10      1762
11     17363
12     12169
13     18754
14      2236
15      9709
16     16539
17     17941
18     14442
19      7699
20     11951
21     15690
22     11558
23      1823
24      6124


In [82]:
# Samples table horizon_id FK 

# Drop old horizon_id if it exists (to avoid confusion)
samples_check = samples_check.drop(columns=['horizon_id'], errors='ignore')

samples_check['sample_id'] = samples_check['sample_id'].astype(str).str.strip().str.replace('.0', '', regex=False)
df_morpho_cleaned['sample_id'] = df_morpho_cleaned['sample_id'].astype(str).str.strip().str.replace('.0', '', regex=False)


# Merge horizon_id from morph into samples
samples_check = samples_check.merge(
    df_morpho_cleaned[['sample_id', 'horizon_id']],
    on='sample_id',
    how='left'
)

missing = samples_check[samples_check['horizon_id'].isna()]
print(f"{len(missing)} samples could not be matched with a horizon_id.")


9704 samples could not be matched with a horizon_id.


In [83]:
# Re-order and select relevant columns
samples_clean1 = samples_check[[
    'sample_id',
    'site_info_id', 
    'profile',
    'horizon_id',
    'year',
    'shelf',
    'room'  # Ensure this matches the column name in your DataFrame
]].copy()

# Preview the result
samples_clean1.head()


,sample_id,site_info_id,profile,horizon_id,year,shelf,room
0,630,172,139,Hb_139/46_1_1,1946.0,1,22
1,631,173,139,NaN,1946.0,1,22
2,632,174,139,Hb_139/46_3_1,1946.0,1,22
3,633,175,139,NaN,1946.0,1,22
4,687,1034,208,Hb_208/46_1_1,1946.0,1,22


In [84]:
samples_clean1.head()

,sample_id,site_info_id,profile,horizon_id,year,shelf,room
0,630,172,139,Hb_139/46_1_1,1946.0,1,22
1,631,173,139,NaN,1946.0,1,22
2,632,174,139,Hb_139/46_3_1,1946.0,1,22
3,633,175,139,NaN,1946.0,1,22
4,687,1034,208,Hb_208/46_1_1,1946.0,1,22


## completing profile_record_clean table

In [85]:
profile_record_clean.head()

,profile_record_id,profile,site_info_id,sample_id,soil_type_id
0,1,139,<NA>,<NA>,<NA>
1,2,139,<NA>,<NA>,<NA>
2,3,139,<NA>,<NA>,<NA>
3,4,139,<NA>,<NA>,<NA>
4,5,208,<NA>,<NA>,<NA>


In [86]:
# Drop old foreign key columns if they exist
profile_record_clean = profile_record_clean.drop(
    columns=['site_info_id', 'sample_id', 'soil_type_id'], errors='ignore'
)

# Merge site_info_id (one-to-one or many-to-one) using 'ID'
profile_record_clean = profile_record_clean.merge(
    site_info_clean[['profile', 'site_info_id']],
    on='ID',
    how='left'
)

# Merge soil_type_id (one-to-one or many-to-one) using 'profile'
profile_record_clean = profile_record_clean.merge(
    soil_type_clean[['profile', 'soil_type_id']],
    on='profile',
    how='left'
)

# Merge sample_id (one-to-one or many-to-one) using 'profile'
profile_record_clean = profile_record_clean.merge(
    samples_clean1[['profile', 'sample_id']],
    on='profile',
    how='left'
)

#drop PK and replace w new profile_record_id PK
# Drop old FK columns if they exist
profile_record_clean = profile_record_clean.drop(
    columns=['profile_record_id' ],
    errors='ignore'
)

# Ensure primary key 'profile_record_id' exists
if 'profile_record_id' not in profile_record_clean.columns:
    profile_record_clean.insert(0, 'profile_record_id', range(1, len(profile_record_clean) + 1))

profile_record_clean['profile_record_id'] = profile_record_clean['profile_record_id'].astype(str)

# Final column order
column_order = [
    'profile_record_id',
    'profile',
    'site_info_id',
    'soil_type_id',
    'sample_id'
]
profile_record_clean = profile_record_clean[column_order]

# Check for missing links (optional sanity check)
missing_site = profile_record_clean[profile_record_clean['site_info_id'].isna()]
missing_soil = profile_record_clean[profile_record_clean['soil_type_id'].isna()]

print(f"{len(missing_site)} profiles missing site_info_id")
print(f"{len(missing_soil)} profiles missing soil_type_id")

profile_record_clean.head()

KeyError: 'ID'

In [118]:
df_morpho_cleaned.head()

,horizon_id,sample_id,profile,horizon_layer,upper_depth,lower_depth,dry_color_name,dry_hue,dry_value,dry_chroma,...,compaction,durability,pore_quantity,pore_diameter,pore_shape,root_quantity,root_diameter,moisture_degree,Unnamed: 46,profile_record_id
0,B_101/62_1_1,10999,101_62,1.0,0.0,11.0,Pardo-acinzentado a pardo,10YR,5.0,2.5,...,Pequena a minima,Brando,Pouco poroso,Muito finos,NaN,Muitas finas e bastantes medias,NaN,Seco,NaN,<NA>
1,B_101/62_2_1,11000,101_62,2.0,11.0,28.0,Pardo,10YR,5.0,3.0,...,Pequena,Brando,Pouco poroso,Muito finos,NaN,Bastantes finas e medias e raras grossas,NaN,Seco,NaN,<NA>
2,B_101/62_3_1,11001,101_62,3.0,28.0,54.0,Pardo-amarelado-claro,10YR,6.0,4.0,...,Pequena a minima,Brando,Pouco poroso,Muito finos,NaN,Algumas finas e medias e raras grossas,NaN,Seco,NaN,<NA>
3,B_101/62_4_1,11002,101_62,4.0,54.0,90.0,Amarelo a amarelo-avermelhado,"8,75YR",7.0,6.0,...,Pequena a minima,Brando,Pouco a medianamente poroso,Muito finos,NaN,"Poucas finas, algumas medias e raras grossas",NaN,Seco,NaN,<NA>
4,B_101/62_5_2,11003,101_62,5.0,90.0,160.0,Pardo-avermelhado,"7,5YR",7.0,6.0,...,Pequena,Brando,Pouco a medianamente poroso,Muito finos,NaN,Raras,Medias e grossas,Seco a humido,NaN,<NA>


In [122]:
# Convert to string dtype in both DataFrames before merge
df_morpho_cleaned['profile_record_id'] = df_morpho_cleaned['profile_record_id'].astype(str)
profile_record_clean['profile_record_id'] = profile_record_clean['profile_record_id'].astype(str)

#drop old FK
# Drop old FK columns if they exist
df_morpho_cleaned = df_morpho_cleaned.drop(
    columns=['profile_record_id'],
    errors='ignore'
)
# Now merge on 'profile_record_id'
df_morpho_cleaned = df_morpho_cleaned.merge(
    profile_record_clean[['profile', 'profile_record_id']],
    on='profile',
    how='left'
)

df_morpho_cleaned1 = df_morpho_cleaned[[
    'horizon_id',
    'sample_id',
    #'profile_record_id',
    'profile',
    'horizon_layer',
    'upper_depth',
    'lower_depth',
    'moisture_degree',
    'root_quantity',
    'root_diameter',
    'texture',
    'structure_type',
    'structure_class',
    'structure_degree',
    'pore_diameter',
    'pore_quantity',
    'pore_shape',
    'dry_color_name',
    'dry_hue',
    'dry_value',
    'dry_chroma',
    'moist_color_name',
    'moist_hue',
    'moist_value',
    'moist_chroma',
    'compaction',
    'durability'
]]

In [123]:
df_morpho_cleaned1.head()



,horizon_id,sample_id,profile,horizon_layer,upper_depth,lower_depth,moisture_degree,root_quantity,root_diameter,texture,...,dry_color_name,dry_hue,dry_value,dry_chroma,moist_color_name,moist_hue,moist_value,moist_chroma,compaction,durability
0,B_101/62_1_1,10999,101_62,1.0,0.0,11.0,Seco,Muitas finas e bastantes medias,NaN,Arenoso,...,Pardo-acinzentado a pardo,10YR,5.0,2.5,Pardo-acinzentado-escuro,10YR,4.0,2.0,Pequena a minima,Brando
1,B_101/62_2_1,11000,101_62,2.0,11.0,28.0,Seco,Bastantes finas e medias e raras grossas,NaN,Arenoso-franco,...,Pardo,10YR,5.0,3.0,Pardo-amarelado-escuro,10YR,3.0,4.0,Pequena,Brando
2,B_101/62_3_1,11001,101_62,3.0,28.0,54.0,Seco,Algumas finas e medias e raras grossas,NaN,Arenoso-franco,...,Pardo-amarelado-claro,10YR,6.0,4.0,Pardo-amarelado-escuro,10YR,4.0,4.0,Pequena a minima,Brando
3,B_101/62_4_1,11002,101_62,4.0,54.0,90.0,Seco,"Poucas finas, algumas medias e raras grossas",NaN,Franco-arenoso a arenoso-franco,...,Amarelo a amarelo-avermelhado,"8,75YR",7.0,6.0,Pardo-forte,"7,5YR",5.0,6.0,Pequena a minima,Brando
4,B_101/62_5_2,11003,101_62,5.0,90.0,160.0,Seco a humido,Raras,Medias e grossas,Arenoso-franco,...,Pardo-avermelhado,"7,5YR",7.0,6.0,Pardo-forte,"7,5YR",5.0,6.0,Pequena,Brando


In [126]:
df_morpho_cleaned1.to_csv('/Users/inesschwartz/Desktop/morpho_clean.csv')

## Completing site_info_clean table

In [42]:
# Convert keys to string before merge to avoid dtype mismatch errors
site_info_clean['site_info_id'] = site_info_clean['ID'].astype(str)
geo_features_clean['ID'] = geo_features_clean['ID'].astype(str)
climate_features_clean['ID'] = climate_features_clean['ID'].astype(str)
topo_features_clean['ID'] = topo_features_clean['ID'].astype(str)

/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_8750/4452788.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  climate_features_clean['ID'] = climate_features_clean['ID'].astype(str)


In [43]:
# Drop old FK columns if they exist
site_info_clean = site_info_clean.drop(
    columns=['geo_features_id', 'climate_id', 'topo_features_id', 'district_id', 'land_cover_id', 'geology_id','topo_feature_id','sampling_date', 'districts_id' ],
    errors='ignore'
)

In [44]:
# --- Step 1: Convert 'ID' columns to string ---
site_info_clean['ID'] = site_info_clean['ID'].astype(str)
geo_features_clean['ID'] = geo_features_clean['ID'].astype(str)
climate_features_clean['ID'] = climate_features_clean['ID'].astype(str)
topo_features_clean['ID'] = topo_features_clean['ID'].astype(str)

# --- Step 2: Merge feature tables into site_info_clean ---
site_info_clean = site_info_clean.merge(
    geo_features_clean[['ID', 'geo_features_id']],
    on='ID',
    how='left'
)

site_info_clean = site_info_clean.merge(
    climate_features_clean[['ID', 'climate_id']],
    on='ID',
    how='left'
)

site_info_clean = site_info_clean.merge(
    topo_features_clean[['ID', 'topo_features_id']],
    on='ID',
    how='left'
)

# --- Step 3: Drop 'ID' column from all involved tables ---
for table in ['site_info_clean', 'geo_features_clean', 'climate_features_clean', 'topo_features_clean']:
    df = globals().get(table)
    if df is not None and 'ID' in df.columns:
        df = df.drop(columns=['ID'])
        globals()[table] = df
        print(f"✅ Dropped 'ID' from {table}")
    else:
        print(f"⏭️ Skipping {table}: not found or no 'ID' column")

# --- District: Skip merging district_id, use district name directly ---

# don't see need for district_id, just use district name
# For district (assuming district columns are already strings)
# site_info_clean = site_info_clean.merge(
#     district_clean[['district', 'district_id']],
#     on='district',
#     how='left'
# )


✅ Dropped 'ID' from site_info_clean
✅ Dropped 'ID' from geo_features_clean
✅ Dropped 'ID' from climate_features_clean
✅ Dropped 'ID' from topo_features_clean


/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_8750/2925909182.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  climate_features_clean['ID'] = climate_features_clean['ID'].astype(str)


In [45]:
site_info_clean.head()

,site_info_id,profile,X_coord,Y_coord,district,geo_features_id,climate_id,topo_features_id
0,2770,1/57,12.161278,-15.222598,Namibe,1,1,1
1,48,1/59,12.575775,-4.866986,Cabinda,2,2,2
2,1618,1/61,15.098840,-11.225411,Cuanza Sul,3,3,3
3,881,1/63,17.081955,-9.274587,Malanje,4,4,4
4,1750,1/64,20.788116,-11.568683,Moxico,5,5,5


In [46]:
## need to drop ID from climate, geo_features, topo_features, and site_info_clean

In [47]:
site_info_clean.head()

,site_info_id,profile,X_coord,Y_coord,district,geo_features_id,climate_id,topo_features_id
0,2770,1/57,12.161278,-15.222598,Namibe,1,1,1
1,48,1/59,12.575775,-4.866986,Cabinda,2,2,2
2,1618,1/61,15.098840,-11.225411,Cuanza Sul,3,3,3
3,881,1/63,17.081955,-9.274587,Malanje,4,4,4
4,1750,1/64,20.788116,-11.568683,Moxico,5,5,5


In [48]:
profile_record_clean.head()

,profile_record_id,profile,site_info_id,soil_type_id,sample_id
0,1,139,<NA>,NaN,630
1,2,139,<NA>,NaN,631
2,3,139,<NA>,NaN,632
3,4,139,<NA>,NaN,633
4,5,139,<NA>,NaN,1817


In [49]:
## changing column headers to be SQL friendly
# Rename problematic columns
merged1 = merged1.rename(columns={
    'atm_1/3': 'atm_1_3',
    'Ca++': 'Ca',
    'Mg++': 'Mg',
    'Na+': 'Na',
    'K+': 'K',
    'Min_<0,002': 'Min_lt_0002',
    'Min_0,05-0,02': 'Min_005_002',
    'Min_0,2-0,05': 'Min_02_005',
    'Min_2-0,2': 'Min_2_02',
    'As': 'arsenic',
    'P': 'phosphorus',
    'S': 'sulfur',
    'V': 'vanadium'
})


# SAVING CLEANED TABLES TO CSV FOR DB

In [50]:
import pandas as pd

# Prepare dictionary of clean tables
tables = {
    "samples": samples_clean1,
    "analyses": merged1,
    "profile_record": profile_record_clean,
    "morphology_horizon": morphology_clean1,
    "soil_type": soil_type_clean,
    "site_info": site_info_clean,
    "climate_feat": climate_features_clean,
    "topo_feat": topo_features_clean,
    "geo_feat": geo_features_clean,
    #"districts": district_clean
}

# # Save each table to CSV with 'NULL' in empty cells
for name, df in tables.items():
#     df.replace("", pd.NA, inplace=True)  # Replace empty strings with NA
    df.to_csv(f"/Users/inesschwartz/GreenDataScience/Thesis/tables_clean/{name}.csv", index=False)

In [51]:
# Save random 100 rows of each table as mini versions for DB test
import os

# Create folder if it doesn't exist
mini_path = "/Users/inesschwartz/GreenDataScience/Thesis/tables_clean_mini"
os.makedirs(mini_path, exist_ok=True)

# Save random 100 rows of each table as mini versions
samples_clean.sample(n=100, random_state=42).to_csv(f"{mini_path}/samples.csv", index=False)
merged1.sample(n=100, random_state=42).to_csv(f"{mini_path}/analyses.csv", index=False)
profile_record_clean.sample(n=100, random_state=42).to_csv(f"{mini_path}/profile_record.csv", index=False)
morphology_clean1.sample(n=100, random_state=42).to_csv(f"{mini_path}/morphology_horizon.csv", index=False)
soil_type_clean.sample(n=100, random_state=42).to_csv(f"{mini_path}/soil_type.csv", index=False)
site_info_clean.sample(n=100, random_state=42).to_csv(f"{mini_path}/site_info.csv", index=False)
climate_features_clean.sample(n=100, random_state=42).to_csv(f"{mini_path}/climate_feat.csv", index=False)
topo_features_clean.sample(n=100, random_state=42).to_csv(f"{mini_path}/topo_feat.csv", index=False)
geo_features_clean.sample(n=100, random_state=42).to_csv(f"{mini_path}/geo_feat.csv", index=False)
#district_clean.sample(n=100, random_state=42).to_csv(f"{mini_path}/districts.csv", index=False)



In [52]:
# List your DataFrame variables here
dataframes = {
    "samples": samples_clean1,
    "analyses": merged1,
    "profile_record": profile_record_clean,
    "morphology_horizon": morphology_clean1,
    "soil_type": soil_type_clean,
    "site_info": site_info_clean,
    "climate_feat": climate_features_clean,
    "topo_feat": topo_features_clean,
    "geo_feat": geo_features_clean
    #"districts": district_clean
}

# Print column names and their datatypes
for name, df in dataframes.items():
    print(f"\n{name} ({len(df)} rows):")
    for col in df.columns:
        dtype = df[col].dtype
        print(f"  - {col} ({dtype})")



samples (14766 rows):
  - sample_id (object)
  - site_info_id (object)
  - profile (object)
  - horizon_id (object)
  - year (float64)
  - shelf (string)
  - room (object)

analyses (7847 rows):
  - lab_sample_id (object)
  - sample_id (object)
  - EG (float64)
  - thick_clay (float64)
  - fine_clay (float64)
  - silt (float64)
  - clay (float64)
  - Eq_Hum (float64)
  - atm_1_3 (float64)
  - atm_15 (float64)
  - CACO3 (float64)
  - gypsum (float64)
  - free_iron (float64)
  - organic_carbon (float64)
  - total_N (float64)
  - P205 (float64)
  - organic_material (float64)
  - pH_H2O (float64)
  - pH_KCL (float64)
  - Ca (float64)
  - Mg (float64)
  - Na (float64)
  - K (float64)
  - exchangable_bases_sum (float64)
  - CEC (float64)
  - vanadium (object)
  - conductivity (float64)
  - soluble_sodium (float64)
  - Min_lt_0002 (object)
  - Min_005_002 (object)
  - Min_02_005 (object)
  - Min_2_02 (object)
  - field_sample_code (object)
  - Depth (object)
  - Al (float64)
  - Si (float64)

In [53]:
missing = profile_record_clean[~profile_record_clean['soil_type_id'].isin(soil_type_clean['soil_type_id'])]
print(missing)

      profile_record_id profile site_info_id  soil_type_id sample_id
0                     1     139         <NA>           NaN       630
1                     2     139         <NA>           NaN       631
2                     3     139         <NA>           NaN       632
3                     4     139         <NA>           NaN       633
4                     5     139         <NA>           NaN      1817
...                 ...     ...          ...           ...       ...
86660             86661  540/67         3627           NaN     18867
86661             86662  540/67         3627           NaN     18868
86662             86663  540/67         3627           NaN     18869
86663             86664  540/67         3627           NaN     18870
86664             86665  538/67         3611           NaN     18871

[51532 rows x 5 columns]


In [54]:
samples_clean1.head()

,sample_id,site_info_id,profile,horizon_id,year,shelf,room
0,630,172,139,Hb_139/46_1_1,1946.0,1,22
1,631,173,139,Hb_139/46_2_1,1946.0,1,22
2,632,174,139,Hb_139/46_3_1,1946.0,1,22
3,633,175,139,NaN,1946.0,1,22
4,687,1034,208,Hb_208/46_1_1,1946.0,1,22


In [55]:
missing1 = site_info_clean[~site_info_clean['site_info_id'].isin(samples_clean['site_info_id'])]
print(missing1)


     site_info_id profile    X_coord    Y_coord    district  geo_features_id  \
0            2770    1/57  12.161278 -15.222598      Namibe                1   
2            1618    1/61  15.098840 -11.225411  Cuanza Sul                3   
4            1750    1/64  20.788116 -11.568683      Moxico                5   
5            3097    1/66  17.666766 -14.655526         NaN                6   
8            2675   10/54  14.445188 -14.922688       Huila                9   
...           ...     ...        ...        ...         ...              ...   
4313         1689   99/63  18.164654 -11.382465     Malanje             4314   
4314         3247   99/66  18.994294 -15.814591         NaN             4315   
4317         1485  99c/63  17.541534 -10.890781     Malanje             4318   
4319         1213   9c/63  16.357859  -9.986349     Malanje             4320   
4320         1505   9c/65  20.937136 -10.942842   Lunda Sul             4321   

      climate_id  topo_features_id  
0 

In [59]:
soil_type_clean.head()

,soil_type_id,profile,CEP_GR,CEP_NAME,FAO
0,1,1/51,NaN,NaN,NaN
1,2,1/57,Aridicos,Aridicos com calcario Pardo-cinzentos,CLha
2,3,1/59,Psamoferralicos,"Psamo-ferralicos Amarelos ou Alaranjados, sedi...",FRxa
3,4,1/63,Ferraliticos,Fracamente Ferralicos Vermelhos Clino-argilico...,FRh
4,5,10/54,Ferraliticos,Fracamente Ferralicos pardo-amarelados,FRh


In [60]:
# Get list of valid soil_type_ids
valid_soil_type_ids = set(soil_type_clean['soil_type_id'])

# Find invalid soil_type_ids in profile_df
invalid_rows = profile_record_clean[~profile_record_clean['soil_type_id'].isin(valid_soil_type_ids)]

# Show the result
if not invalid_rows.empty:
    print("❌ These rows reference soil_type_id values that do NOT exist in soil_type.csv:")
    print(invalid_rows)
else:
    print("✅ All soil_type_id values in profile_record.csv are valid.")


❌ These rows reference soil_type_id values that do NOT exist in soil_type.csv:
      profile_record_id profile site_info_id  soil_type_id sample_id
0                     1     139         <NA>           NaN       630
1                     2     139         <NA>           NaN       631
2                     3     139         <NA>           NaN       632
3                     4     139         <NA>           NaN       633
4                     5     139         <NA>           NaN      1817
...                 ...     ...          ...           ...       ...
86660             86661  540/67         3627           NaN     18867
86661             86662  540/67         3627           NaN     18868
86662             86663  540/67         3627           NaN     18869
86663             86664  540/67         3627           NaN     18870
86664             86665  538/67         3611           NaN     18871

[51532 rows x 5 columns]


In [61]:
profile_record_clean.head()

,profile_record_id,profile,site_info_id,soil_type_id,sample_id
0,1,139,<NA>,NaN,630
1,2,139,<NA>,NaN,631
2,3,139,<NA>,NaN,632
3,4,139,<NA>,NaN,633
4,5,139,<NA>,NaN,1817


In [62]:
# Find rows where soil_type_id is null in profile_record_clean
null_rows = profile_record_clean[profile_record_clean['site_info_id'].isnull()]

# Show the result
if not null_rows.empty:
    print("❌ These rows have NULL values in the site_info_id column of profile_record.csv:")
    print(null_rows)
else:
    print("✅ No NULL values found in the site_info_id column of profile_record.csv.")


❌ These rows have NULL values in the site_info_id column of profile_record.csv:
      profile_record_id profile site_info_id  soil_type_id sample_id
0                     1     139         <NA>           NaN       630
1                     2     139         <NA>           NaN       631
2                     3     139         <NA>           NaN       632
3                     4     139         <NA>           NaN       633
4                     5     139         <NA>           NaN      1817
...                 ...     ...          ...           ...       ...
86240             86241  508/71         <NA>           NaN     18765
86267             86268  516/68         <NA>           NaN     18772
86268             86269  516/69         <NA>           NaN     18773
86270             86271  493/68         <NA>           NaN     18781
86271             86272  493/69         <NA>           NaN     18782

[29884 rows x 5 columns]


In [63]:
# Find rows where soil_type_id is null in profile_record_clean
null_rows = profile_loc[profile_loc['ID'].isnull()]

# Show the result
if not null_rows.empty:
    print("❌ These rows have NULL values in the ID column of perfis_local.csv:")
    print(null_rows)
else:
    print("✅ No NULL values found in the ID column of perfis_local.csv.")


✅ No NULL values found in the ID column of perfis_local.csv.


In [64]:
soil_type_ids = set(soil_type_clean['soil_type_id'].unique())
profile_soil_type_ids = set(profile_record_clean['soil_type_id'].unique())

missing_ids = profile_soil_type_ids - soil_type_ids

print("soil_type_ids in profile_record missing from soil_type.csv:", missing_ids)


soil_type_ids in profile_record missing from soil_type.csv: {nan}


In [65]:
# Check if 4835 is in soil_type_id column of soil_type.csv
soil_type_has_4835 = 4835 in soil_type_clean['soil_type_id'].values

# Check if 4835 is in soil_type_id column of profile_record.csv
profile_record_has_4835 = 4835 in profile_record_clean['soil_type_id'].values

print(f"4835 in soil_type.csv? {soil_type_has_4835}")
print(f"4835 in profile_record.csv? {profile_record_has_4835}")

4835 in soil_type.csv? False
4835 in profile_record.csv? False


In [66]:
print(4835 in profile_record_clean['soil_type_id'].values)

False
